In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb

#use a class to define your input and output in order to have mutable variables
class gfc_process_IO:
    def __init__(self):
        #set up your inputs
        self.threshold = 30
        
        #set up output
        self.visualization = False
        self.clip_map = None
        self.csv_file = None
        
gfc_process_io = gfc_process_IO()

In [2]:
GFC_MESSAGE = 'Click on "Update" to visualize the data'
EXPORT_MESSAGE = 'Click on "Export" to export the data'
NO_THRESHOLD = 'No threshold have been selected'
NO_AOI = 'No aoi have been provided, select one in step 1'
NO_VIZ= 'Befor launching the export, it is strongly advised to vizualize data'

In [3]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "gfc_process_widget"
title = "Results"

gfc_results = wf.Tile(id_, title, inputs=['No results to display yet'])

In [4]:
import ipyvuetify as v

#create the output alert 
gfc_process_output = wf.OutputWidget(GFC_MESSAGE)

#create the button that start your process
gfc_process_btn = wf.ProcessBtn('Update data')

#create the widgets following ipyvuetify requirements 
gfc_process_slider = v.Slider(
    label= 'Threshold', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=30,
)

gfc_process_inputs = [gfc_process_slider]

#bind the widget to the inputs
wb.bind(gfc_process_slider, gfc_process_io, 'threshold', gfc_process_output)

#create a process tile
id_ = "gfc_process_widget"
title = 'gfc_Process'

gfc_process = wf.Tile(
    id_, 
    title, 
    btn=gfc_process_btn, 
    inputs=gfc_process_inputs, 
    output=gfc_process_output
)

#bind the button to the process by writing a custom function
from scripts import run
from sepal_ui.scripts import utils
from functools import partial

def process_start(widget, event, data, output):
    
    #in order to write the results
    global gfc_results
    
    #debug
    asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    
    #asset = gfc_aoi_IO.assetId
    threshold = gfc_process_io.threshold
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check asset 
    if not asset: 
        utils.displayIO(output, NO_AOI, alert_type='error')
        utils.toggleLoading(widget)
        return 
    
    if not threshold:
        utils.displayIO(output, NO_THRESHOLD, alert_type='error')
        utils.toggleLoading(widget)
        return
    
    #display the results 
    result_layout = run.displayGfcResults(asset, threshold, output)
    gfc_results.children[0].children = [gfc_results.children[0].children[0]] + result_layout
    
    #validate the visualization process
    setattr(gfc_process_io, 'visualization', True)
    
    #toggle the loading button
    utils.toggleLoading(widget)

gfc_process_btn.on_event('click', partial(process_start, output=gfc_process_output))

In [5]:
#create the output alert 
gfc_export_output = wf.OutputWidget(EXPORT_MESSAGE)

#create the button that start your process
gfc_export_btn = wf.ProcessBtn('Export data')

#create a process tile
id_ = "gfc_process_widget"
title = 'gfc_export'

gfc_export = wf.Tile(
    id_, 
    title, 
    btn=gfc_export_btn,
    output=gfc_export_output
)

#bind the button to the process by writing a custom function
def export_start(widget, event, data, output):
    
    #in order to write the results
    global gfc_results
    
    #debug
    asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    #visualization = False
    
    #asset = gfc_aoi_IO.assetId
    threshold = gfc_process_io.threshold
    visualization = gfc_process_io.visualization
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check input 
    if not asset: 
        utils.displayIO(output, NO_AOI, alert_type='error')
        utils.toggleLoading(widget)
        return 
    
    if not threshold:
        utils.displayIO(output, NO_THRESHOLD, alert_type='error')
        utils.toggleLoading(widget)
        return
    
    if not visualization: 
        utils.displayIO(output, NO_VIZ, alert_type='warning')
        utils.toggleLoading(widget)
        return 
    
    #launch any process you want
    clip_map, csv_file = run.gfc_analysis(asset, gfc_process_io.threshold, output)
    
    setattr(gfc_process_io, 'clip_map', clip_map)
    setattr(gfc_process_io, 'csv_file', csv_file)
    
    #create the links
    gfc_download_csv = wf.DownloadBtn('GFC hist values in .csv', path=csv_file)
    gfc_download_tif = wf.DownloadBtn('GFC raster in .tif', path=clip_map)
    
    buttons_row = v.Layout(Row=True, xs12=True, children=[
            gfc_download_csv,
            gfc_download_tif,
        ])
    
    #replace the disabled buttons links 
    gfc_results.children[0].children[1].children[0].href = gfc_download_csv.href
    gfc_results.children[0].children[1].children[1].href = gfc_download_tif.href
    
    #enable them
    gfc_results.children[0].children[1].children[0].disabled = False
    gfc_results.children[0].children[1].children[1].disabled = False
    
    #toggle the loading button
    utils.toggleLoading(widget)

gfc_export_btn.on_event('click', partial(export_start, output=gfc_export_output))

In [6]:
#this tiles will only be displayed if you launch voila from this file 
gfc_process

Layout(align_center=True, children=[Card(children=[Html(children=['gfc_Process'], tag='h2'), Flex(children=[La…

In [7]:
#this tiles will only be displayed if you launch voila from this file 
gfc_results

Layout(align_center=True, children=[Card(children=[Html(children=['Results'], tag='h2'), Flex(children=[Layout…

In [8]:
#this tiles will only be displayed if you launch voila from this file 
gfc_export

Layout(align_center=True, children=[Card(children=[Html(children=['gfc_export'], tag='h2'), Flex(children=[Lay…

In [9]:
print(gfc_results.children[0].children[1].children[0])

Layout(align_center=True, children=[Flex(children=['No results to display yet'], xs12=True)], class_='pa-5', row=True)
